In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = tf.convert_to_tensor(x_train, dtype = tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype = tf.int32)

x_test = tf.convert_to_tensor(x_test, dtype = tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype = tf.int32)

x_train = x_train/255.0
x_train = tf.expand_dims(x_train,axis = -1)

x_test = x_test/255.0
x_test = tf.expand_dims(x_test,axis = -1)

11490434/11490434 [==============================] - 0s 0us/step


In [7]:
my_model = tf.keras.models.Sequential([

  Conv2D(filters = 6, kernel_size = 8, activation = 'relu'),
  MaxPooling2D(pool_size =(2,2)),
  Conv2D(filters = 15, kernel_size = 4, activation = 'relu'),
  MaxPooling2D(pool_size =(2,2)),

  Flatten(),

  Dense(128, activation = 'relu'),
  Dense(10, activation = 'softmax')

])

In [8]:
my_model.compile(optimizer = 'adam',
                 loss = 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])

my_model.fit(x_train, y_train, epochs = 3, batch_size = 1024)

my_model.evaluate(x_test, y_test, verbose = 2)

Epoch 1/3
59/59 [==============================] - 21s 325ms/step - loss: 1.2421 - accuracy: 0.6794
Epoch 2/3
59/59 [==============================] - 21s 352ms/step - loss: 0.3747 - accuracy: 0.8890
Epoch 3/3
59/59 [==============================] - 20s 336ms/step - loss: 0.2749 - accuracy: 0.9183
313/313 - 2s - loss: 0.2295 - accuracy: 0.9322 - 2s/epoch - 6ms/step


[0.2294807881116867, 0.932200014591217]

In [9]:
BATCH_SIZE = 1024

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [10]:
from tensorflow.keras.layers import Layer

class Linear(Layer):
  """y = Wx + b"""

  def __init__(self, units = 32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.W = self.add_weight(shape = (input_shape[-1],self.units),
                             initializer = 'random_normal',trainable = True)
    self.b = self.add_weight(shape = (self.units,),
                             initializer = 'random_normal', trainable = True)

  def call(self, inputs):
    return tf.matmul(inputs, self.W) + self.b

In [26]:
class DoubleDense(Layer):
  """Linear-relu + Linear-Softmax"""

  def __init__(self, nb_classes):
    super(DoubleDense, self).__init__()
    self.nb_classes = nb_classes

  def build(self, input_shape):
    self.linear_1 = Linear(units = 128)
    self.linear_2 = Linear(units = self.nb_classes)

  def call(self, inputs):
    x = tf.nn.relu(self.linear_1(inputs))
    x = tf.nn.softmax(self.linear_2(x))
    return x

In [12]:
class ConvPool2D(Layer):
  """Conv2D-relu + MaxPooling2D"""

  def __init__(self, nb_kernels, kernel_size):
    super(ConvPool2D, self).__init__()
    self.nb_kernels = nb_kernels
    self.kernel_size = kernel_size

  def build(self, input_shape):
    self.conv_2D = Conv2D(filters = self.nb_kernels,
                          kernel_size = self.kernel_size,
                          activation = 'relu')
    self.pool_2D = MaxPooling2D(pool_size = (2,2))

  def call(self, inputs):
    x = self.conv_2D(inputs)
    x = self.pool_2D(x)
    return x

In [22]:
from tensorflow.keras import Model

class MyModel(Model):
  def __init__(self, nb_classes):
    super(MyModel, self).__init__()
    self.nb_classes = nb_classes

  def build(self, input_shape):
    self.conv_pool_1 = ConvPool2D(nb_kernels = 6, kernel_size = 8)
    self.conv_pool_2 = ConvPool2D(nb_kernels = 15, kernel_size = 4)
    self.flatten = Flatten()
    self.double_dense = DoubleDense(nb_classes = self.nb_classes)

  def call(self, inputs):
    self.x_0 = self.conv_pool_1(inputs)
    self.x_1 = self.conv_pool_2(self.x_0)
    self.x_2 = self.flatten(self.x_1)
    self.predictions = self.double_dense(self.x_2)
    return self.predictions

In [34]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam()

my_model = MyModel(nb_classes = 10)
my_model.compile(optimizer, loss_function)

In [33]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy =tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy =tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [17]:
x= tf.Variable(3.0)
with tf.GradientTape() as tape:
  y = tf.square(x)
dy_dx = tape.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [31]:
def train_step(images,labels):

  with tf.GradientTape() as tape:

    predictions = my_model(images)
    loss = loss_function(labels, predictions)
    gradients = tape.gradient(loss, my_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,my_model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

def test_step(images, labels):
  predictions = my_model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [35]:
import time
my_model = MyModel(nb_classes = 10)
start = time.time()

for epoch in range(3):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch{:0f}, Loss:{:.3f}, Accuracy:{:.3f}' + 'Test Loss:{:.3f}, Test Accuracy:{:.3f}'
  print(template.format(epoch + 1,train_loss.result(), train_accuracy.result()*100,
                        test_loss.result(), test_accuracy.result()*100))

  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

end = time.time()
print('Time = ', end - start)

Epoch1.000000, Loss:1.387, Accuracy:66.175Test Loss:0.471, Test Accuracy:85.900
Epoch2.000000, Loss:0.403, Accuracy:87.932Test Loss:0.301, Test Accuracy:91.100
Epoch3.000000, Loss:0.287, Accuracy:91.563Test Loss:0.228, Test Accuracy:93.330
Time =  126.71333193778992
